# TSC 标准 Unsloth GRPO 训练（两大场景）

使用标准 Unsloth GRPOTrainer + 离线 Dataset + Reward Function 回溯 SUMO 评估

## 1. 环境配置

In [1]:
import os
import sys

os.environ["UNSLOTH_USE_MODELSCOPE"] = "1"
os.environ["UNSLOTH_VLLM_STANDBY"] = "1"
os.environ["MODELSCOPE_CACHE"] = "model"
os.environ["HF_HOME"] = "model"
print("环境变量已设置")

环境变量已设置


## 1.5 生成/检查 Dataset（可选）

如果 dataset 不存在，此 cell 会自动生成；如果已存在，则跳过。

In [2]:
from generate_grpo_dataset import main as generate_main, CONFIG

DATASET_PATH = "grpo_dataset_two_scenarios"

if not os.path.isdir(DATASET_PATH):
    print(f"⚠️ Dataset 不存在: {DATASET_PATH}")
    print("开始生成 dataset...")

    # 快速验证模式：取消下面注释以使用小规模dataset
    # QUICK_VERIFY = True
    QUICK_VERIFY = False  # 正式训练设为 False

    CONFIG.update({
        "output_dir": DATASET_PATH,
        "state_dir": "grpo_states_two_scenarios",
        "dataset_mode": "two_scenarios",
        "steps_per_tl_signal_step": 100,   # 每TL生成100个signal_step样本
        "steps_per_tl_extend_decision": 100,  # 每TL生成100个extend_decision样本
        "decision_lead_sec": 10,
        "phase_duration_scale_range": (0.7, 1.3),
        "extend_min_green_range": (5, 20),
        "extend_max_green_range": (25, 120),
        "extend_wait_time_range": (5, 25),
        "max_tl_per_scenario": 10,  # 每场景最多10个TL
        "num_workers": 4 if QUICK_VERIFY else 16,
    })

    print("当前配置:")
    print(f"  - 模式: {'快速验证' if QUICK_VERIFY else '正式训练'}")
    print(f"  - warmup_steps: {CONFIG['warmup_steps']}")
    print(f"  - steps_per_tl_signal_step: {CONFIG['steps_per_tl_signal_step']}")
    print(f"  - steps_per_tl_extend_decision: {CONFIG['steps_per_tl_extend_decision']}")
    print(f"  - max_tl_per_scenario: {CONFIG['max_tl_per_scenario']}")
    print(f"  - num_workers: {CONFIG['num_workers']}")

    generate_main()
else:
    print(f"✓ Dataset 已存在: {DATASET_PATH}")

⚠️ Dataset 不存在: grpo_dataset_two_scenarios
开始生成 dataset...
当前配置:
  - 模式: 正式训练
  - warmup_steps: 80
  - steps_per_tl_signal_step: 100
  - steps_per_tl_extend_decision: 100
  - max_tl_per_scenario: 10
  - num_workers: 16
发现场景数: 1
总训练组合数: 10
  优先场景: 0
  其他场景: 10
  并行 workers: 16

开始并行生成（16 workers）...
DEBUG: 使用的 SUMO 路径是: /usr/share/sumo/bin/sumoDEBUG: 使用的 SUMO 路径是: /usr/share/sumo/bin/sumoDEBUG: 使用的 SUMO 路径是: /usr/share/sumo/bin/sumoDEBUG: 使用的 SUMO 路径是: /usr/share/sumo/bin/sumoDEBUG: 使用的 SUMO 路径是: /usr/share/sumo/bin/sumoDEBUG: 使用的 SUMO 路径是: /usr/share/sumo/bin/sumoDEBUG: 使用的 SUMO 路径是: /usr/share/sumo/bin/sumoDEBUG: 使用的 SUMO 路径是: /usr/share/sumo/bin/sumoDEBUG: 使用的 SUMO 路径是: /usr/share/sumo/bin/sumoDEBUG: 使用的 SUMO 路径是: /usr/share/sumo/bin/sumo






Starting SUMO with command: /usr/share/sumo/bin/sumo -c /root/SCU_TSC/sumo_simulation/environments/chengdu/chengdu.sumocfg --step-length 1.0 --no-warnings true --start --device.rerouting.probability 0 on port 18143 (Attempt 1/10)

Starting SUM

Saving the dataset (0/1 shards):   0%|          | 0/1709 [00:00<?, ? examples/s]

✓ Dataset 已保存到: grpo_dataset_two_scenarios

=== Dataset 统计 ===
总样本数: 1709
场景数: 1
信号灯数: 10


## 1.8 Generate Synthetic SFT Dataset

Based on the GRPO dataset, generate synthetic responses to create an SFT dataset.

In [ ]:
import os
import json
import random
import re
from datasets import load_from_disk, Dataset

def extract_json_content(text, marker):
    pattern = f"【{marker}】(.*?)【/{marker}】"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return json.loads(match.group(1))
    return None

def generate_synthetic_response(messages):
    user_content = messages[-1]['content']
    
    # Check task type
    if "【signal_step_input_json】" in user_content:
        data = extract_json_content(user_content, "signal_step_input_json")
        if data:
            # Heuristic for signal_step
            scenario = data.get("scenario", {})
            phase_ids = scenario.get("phase_ids", [1, 3])
            
            # Simple heuristic: pick valid phase, valid time
            # Ideally pick a random one to prevent bias to one phase
            next_phase_id = random.choice(phase_ids)
            green_sec = random.randint(10, 60)
            
            return json.dumps({
                "next_phase_id": next_phase_id,
                "green_sec": green_sec
            })
            
    elif "【extend_decision_input_json】" in user_content:
        data = extract_json_content(user_content, "extend_decision_input_json")
        if data:
            # Heuristic for extend_decision
            # Simple heuristic: 50% yes/no
            should_extend = random.choice(["是", "否"])
            extend_sec = 0
            if should_extend == "是":
                extend_sec = random.randint(5, 30)
                
            return json.dumps({
                "extend": should_extend,
                "extend_sec": extend_sec
            })
            
    return None

def main():
    INPUT_PATH = "grpo_dataset_two_scenarios"
    OUTPUT_PATH = "sft_dataset_synthetic"
    
    if not os.path.exists(INPUT_PATH):
        print(f"Error: {INPUT_PATH} not found.")
        return

    print(f"Loading dataset from {INPUT_PATH}...")
    dataset = load_from_disk(INPUT_PATH)
    
    # Limit dataset to 2000 samples for SFT (doesn't need too much data)
    MAX_SFT_SAMPLES = 2000
    if len(dataset) > MAX_SFT_SAMPLES:
        print(f"Dataset has {len(dataset)} samples, limiting to {MAX_SFT_SAMPLES} for SFT...")
        indices = random.sample(range(len(dataset)), MAX_SFT_SAMPLES)
        dataset = dataset.select(indices)
    
    new_data = []
    
    print("Generating synthetic responses...")
    for item in dataset:
        prompt_messages = item['prompt'] # List of {role, content}
        
        response_json = generate_synthetic_response(prompt_messages)
        
        if response_json:
            # Create full conversation for SFT
            # Clone messages to avoid modifying original reference if any
            new_messages = [m.copy() for m in prompt_messages]
            new_messages.append({
                "role": "assistant",
                "content": response_json
            })
            
            new_data.append({
                "messages": new_messages
            })
    
    if not new_data:
        print("No valid samples generated!")
        return
        
    print(f"Generated {len(new_data)} samples.")
    
    # Create new dataset
    sft_dataset = Dataset.from_list(new_data)
    sft_dataset.save_to_disk(OUTPUT_PATH)
    print(f"Saved synthetic SFT dataset to {OUTPUT_PATH}")

    # Verify one sample
    print("\nSample 0:")
    print(json.dumps(sft_dataset[0]["messages"][-1], ensure_ascii=False, indent=2))

if __name__ == "__main__":
    main()


## 1.9 SFT Training

Train the model on the synthetic SFT dataset before GRPO.

In [ ]:

import os
import torch
from unsloth import FastLanguageModel
from trl import SFTConfig, SFTTrainer
from datasets import load_from_disk
from unsloth.chat_templates import get_chat_template
from transformers import EarlyStoppingCallback

# ==================== Config ====================
max_seq_length = 1024
lora_rank = 32
# model_name = "rd211/Qwen3-0.6B-Instruct"
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
output_dir = "checkpoints/sft_tsc_synthetic"
dataset_path = "sft_dataset_synthetic"

# ==================== Load Model ====================
print(f"Loading model: {model_name}")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=False,
    fast_inference=False,
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.7,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=lora_rank * 2,
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)
model.config.use_cache = False

# ==================== Load Dataset ====================
if not os.path.isdir(dataset_path):
    raise FileNotFoundError(f"Dataset not found: {dataset_path}. Run generate_synthetic_sft_dataset.py first.")

dataset = load_from_disk(dataset_path)
print(f"Loaded {len(dataset)} samples for SFT.")

# Split dataset into train and test
dataset = dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# Limit eval dataset to 200 samples for speed
if len(eval_dataset) > 500:
    eval_dataset = eval_dataset.select(range(500))

print(f"Train samples: {len(train_dataset)}")
print(f"Eval samples: {len(eval_dataset)}")

# ==================== Configure Trainer ====================
# Apply chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="qwen-2.5", 
)

# Formatting function for chat 
def formatting_prompts_func(examples):
    convos = examples["messages"]
    
    # Logic to handle both batched and non-batched inputs
    if isinstance(convos, list) and len(convos) > 0 and isinstance(convos[0], dict):
        # Single conversation (list of dicts)
        texts = [tokenizer.apply_chat_template(convos, tokenize=False, add_generation_prompt=False)]
    else:
        # Batch of conversations (list of lists of dicts)
        texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
        
    return texts

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False, 
    formatting_func=formatting_prompts_func,
    args=SFTConfig(
        output_dir=output_dir,
        num_train_epochs=3, # Increase epochs relying on early stopping
        per_device_train_batch_size=2,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=4,
        learning_rate=2e-4, 
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        fp16_full_eval=not torch.cuda.is_bf16_supported(),
        bf16_full_eval=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        eval_strategy="steps",
        eval_steps=30, # Evaluate every 30 steps
        save_strategy="steps",
        save_steps=30,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
    ),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)] # Stop if no improvement for 3 eval steps (90 steps)
)


print("Starting SFT Training with Early Stopping...")
trainer.train()

print(f"Saving model to {output_dir}")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("Done.")


# Clean up memory for the next stage
import gc
try:
    del model, tokenizer, trainer
except NameError:
    pass
gc.collect()
torch.cuda.empty_cache()
print("Memory cleaned up for GRPO stage.")


## 2. 加载模型

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
lora_rank = 32

os.environ["HF_HOME"] = "model"
os.environ["MODELSCOPE_CACHE"] = "model"

# BASE_MODEL_DIR = "model/models/qwen3-4B-SFT"
BASE_MODEL_DIR = "Qwen/Qwen2.5-0.5B-Instruct"
# BASE_MODEL_DIR = "rd211/Qwen3-0.6B-Instruct"
CHECKPOINT_DIR = "checkpoints/grpo_tsc_two_scenarios_latest"


def _looks_like_checkpoint(path: str) -> bool:
    if not os.path.isdir(path):
        return False
    marker_files = [
        "adapter_config.json",
        "adapter_model.safetensors",
        "adapter_model.bin",
        "config.json",
    ]
    return any(os.path.isfile(os.path.join(path, f)) for f in marker_files)



SFT_CHECKPOINT_DIR = "checkpoints/sft_tsc_synthetic"

resume_from = CHECKPOINT_DIR if _looks_like_checkpoint(CHECKPOINT_DIR) else BASE_MODEL_DIR

if _looks_like_checkpoint(CHECKPOINT_DIR):
    print(f"✓ 从 checkpoint 继续训练: {CHECKPOINT_DIR}")
    resume_from = CHECKPOINT_DIR
elif _looks_like_checkpoint(SFT_CHECKPOINT_DIR):
    print(f"✓ 从 SFT 模型开始训练: {SFT_CHECKPOINT_DIR}")
    resume_from = SFT_CHECKPOINT_DIR
else:
    print(f"ℹ 从基础模型开始: {BASE_MODEL_DIR}")
    resume_from = BASE_MODEL_DIR

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=resume_from,
    max_seq_length=max_seq_length,
    load_in_4bit=False,
    fast_inference=True,
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.8,
)

if resume_from == BASE_MODEL_DIR:
    model = FastLanguageModel.get_peft_model(
        model,
        r=lora_rank,
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj",
        ],
        lora_alpha=lora_rank * 2,
        use_gradient_checkpointing="unsloth",
        random_state=3407,
    )
else:
    try:
        model.gradient_checkpointing_enable()
    except Exception:
        pass
    _trainable = [p for p in model.parameters() if p.requires_grad]
    if len(_trainable) == 0:
        for name, p in model.named_parameters():
            if "lora" in name.lower():
                p.requires_grad = True
        print("⚠️ checkpoint 未检测到可训练参数，已强制启用 LoRA 参数训练")

# Fix for GRPO generation: Must use left padding (applies to ALL cases)
tokenizer.padding_side = "left"
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Apply the same chat template as SFT training
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="qwen-2.5",  # Must match SFT training template
)
print("✓ Applied qwen-2.5 chat template to match SFT training")

# Ensure BOS token is preserved - this is critical for generation
if tokenizer.bos_token_id is None:
    # For Qwen, <|im_start|> acts as BOS in chat context
    tokenizer.bos_token_id = tokenizer.convert_tokens_to_ids("<|im_start|>")
    print(f"设置 bos_token_id: {tokenizer.bos_token_id}")

print(f"Tokenizer padding_side: {tokenizer.padding_side}")
print(f"Tokenizer pad_token_id: {tokenizer.pad_token_id}")
print(f"Tokenizer bos_token_id: {tokenizer.bos_token_id}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 01-19 22:58:59 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
✓ 从 SFT 模型开始训练: checkpoints/sft_tsc_synthetic


INFO:unsloth_zoo.log: Unsloth: Patching vLLM
INFO:unsloth_zoo.log: Unsloth: Patching vLLM to enable standby.
INFO:unsloth_zoo.log: Unsloth: Enabling vLLM standby mode


INFO 01-19 22:59:02 [vllm_utils.py:693] Unsloth: Patching vLLM v1 graph capture
Unsloth: Could not patch vLLM V0 graph capture: No module named 'vllm.worker.model_runner'
==((====))==  Unsloth 2025.10.5: Fast Qwen2 patching. Transformers: 4.56.2. vLLM: 0.11.0.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 23.559 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


INFO:unsloth_zoo.log: Unsloth: Standby mode is enabled. Increasing `gpu_memory_utilization` to 0.9.


Unsloth: vLLM loading model/models/unsloth/Qwen2___5-0___5B-Instruct with actual GPU utilization = 87.63%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 23.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 288.
Unsloth: vLLM's KV Cache can use up to 19.69 GB. Also swap space = 6 GB.
WARNING 01-19 22:59:07 [compilation.py:475] full_cuda_graph is deprecated, use cudagraph_mode=FULL instead.
Unsloth: Not an error, but `device` is not supported in vLLM. Skipping.
INFO 01-19 22:59:07 [utils.py:233] non-default args: {'dtype': torch.bfloat16, 'seed': 0, 'max_model_len': 2048, 'enable_prefix_caching': True, 'swap_space': 6, 'gpu_memory_utilization': 0.8762611176024063, 'max_num_batched_tokens': 2048, 'max_num_seqs': 288, 'max_logprobs': 0, 'disable_log_stats': True, 'enable_lora': True, 'max_lora_rank': 32, 'enable_chunked_prefill': True, 'compilation_config': {"level":3,"debug_dump_path":"","cache_dir":"","backend":"inductor","custom_

...

INFO 01-19 22:59:20 [core.py:210] init engine (profile, create kv cache, warmup model) took 10.71 seconds
INFO 01-19 22:59:21 [llm.py:306] Supported_tasks: ('generate',)


Some weights of Qwen2ForCausalLM were not initialized from the model checkpoint at model/models/unsloth/Qwen2___5-0___5B-Instruct and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'q_norm', 'post_layernorm', 'norm1', 'input_layernorm', 'attention_norm', 'post_attention_layernorm', 'pre_feedforward_layernorm', 'layer_norm1', 'ffn_norm', 'k_norm', 'layer_norm2', 'norm2']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'q_norm', 'post_layernorm', 'norm1', 'input_layernorm', 'attention_norm', 'cross_attn_input_layernorm', 'cross_attn_post_attention_layernorm', 'post_attention_layernorm', 'pre_feedforward_layernorm', 'layer_norm1', 'ffn_norm', 'k_norm', 'layer_norm2', 'norm2']


Unsloth 2025.10.5 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


✓ Applied qwen-2.5 chat template to match SFT training
设置 bos_token_id: 151644
Tokenizer padding_side: left
Tokenizer pad_token_id: 151654
Tokenizer bos_token_id: 151644


## 3. 加载 Dataset

In [4]:
from datasets import load_from_disk
from datasets import Dataset

DATASET_PATH = "grpo_dataset_two_scenarios"
EVAL_TEST_SIZE = 0.02
EVAL_MAX_SAMPLES = 64
EVAL_SEED = 42

if not os.path.isdir(DATASET_PATH):
    raise FileNotFoundError(
        f"Dataset 不存在: {DATASET_PATH}\n"
        "请先运行 generate_grpo_dataset.py 生成离线 dataset"
    )

dataset = load_from_disk(DATASET_PATH)
print(f"✓ Dataset 加载成功: {DATASET_PATH}")
print(f"样本数: {len(dataset)}")

# Split out a small eval set to track real progress.
# Prefer stratified split by task_type so both tasks appear in eval.
try:
    if isinstance(dataset, Dataset) and ("task_type" in dataset.column_names):
        try:
            # Try stratified split first
            split = dataset.train_test_split(
                test_size=EVAL_TEST_SIZE,
                seed=EVAL_SEED,
                stratify_by_column="task_type",
            )
        except Exception as strat_err:
            # Fallback to non-stratified split if stratification fails
            print(f"⚠️ Stratified split failed ({strat_err}), using random split")
            split = dataset.train_test_split(test_size=EVAL_TEST_SIZE, seed=EVAL_SEED)
    else:
        split = dataset.train_test_split(test_size=EVAL_TEST_SIZE, seed=EVAL_SEED)

    train_dataset = split["train"]
    eval_dataset = split["test"]
    if len(eval_dataset) > EVAL_MAX_SAMPLES:
        eval_dataset = eval_dataset.select(range(EVAL_MAX_SAMPLES))

    print(f"✓ Train/Eval split: train={len(train_dataset)}, eval={len(eval_dataset)}")
    if "task_type" in train_dataset.column_names:
        from collections import Counter
        print("  - eval task_type counts:", Counter(eval_dataset["task_type"]))
except Exception as e:
    print(f"⚠️ Train/Eval split failed completely: {e}")
    train_dataset = dataset
    eval_dataset = None

# NOTE: Do NOT apply chat template here!
# GRPOTrainer expects prompt to be a list of messages, not a formatted string.
# It will apply the chat template internally using processing_class (tokenizer).
print("示例 Prompt (message list):")
print(dataset[0]["prompt"])

✓ Dataset 加载成功: grpo_dataset_two_scenarios
样本数: 1709
⚠️ Stratified split failed (Stratifying by column is only supported for ClassLabel column, and column task_type is Value.), using random split
✓ Train/Eval split: train=1674, eval=35
  - eval task_type counts: Counter({'signal_step': 20, 'extend_decision': 15})
示例 Prompt (message list):
[{'content': 'You are a traffic signal control expert. Output only valid JSON without any explanation.', 'role': 'system'}, {'content': '你是交通信号控制优化专家。【signal_step_input_json】{"crossing_id":399226822,"as_of":"2026-01-19 22:57:47","scenario":{"phase_ids":[1,3],"phase_lane_map":{"1":["441579299#14_1","441579299#14_0","441579299#14_2","441579299#14_3"],"3":["476664219#4_1","476664219#4_0"]}},"state":{"current_phase_id":1,"current_phase_elapsed_sec":11,"current_phase_planned_green_sec":11,"phase_metrics_now":[{"phase_id":1,"avg_queue_veh":0.0,"avg_passed_veh_in_current_green":4.0},{"phase_id":3,"avg_queue_veh":0.0,"avg_passed_veh_in_current_green":0.0}]}}【

## 4. 导入 Reward Function

In [5]:
from tsc_reward_function import (
    tsc_reward_fn,
    cleanup_global_pool,
    reward_diag_snapshot,
    reward_diag_last,
)
from transformers import TrainerCallback


class RewardDiagnosticsCallback(TrainerCallback):
    def __init__(self, kl_spike_threshold: float = 5.0):
        self.kl_spike_threshold = float(kl_spike_threshold)

    def on_log(self, args, state, control, logs=None, **kwargs):
        if not logs:
            return
        if not getattr(state, "is_world_process_zero", True):
            return

        # Periodic window summary (aligned to logging)
        if getattr(args, "logging_steps", None) and state.global_step % int(args.logging_steps) == 0:
            snap = reward_diag_snapshot(reset=True)
            total = snap.get("window_total", 0)
            invalid = snap.get("window_invalid", 0)
            rate = (invalid / total) if total else 0.0
            print(
                f"[reward_diag] steps {snap.get('window_start_step')}..{state.global_step} "
                f"invalid_rate={rate:.3f} ({invalid}/{total})"
            )
            by_task_total = snap.get("window_total_by_task", {}) or {}
            by_task_invalid = snap.get("window_invalid_by_task", {}) or {}
            by_reason = snap.get("window_reason_by_task", {}) or {}
            for task, t_total in sorted(by_task_total.items()):
                t_invalid = int(by_task_invalid.get(task, 0))
                t_rate = (t_invalid / t_total) if t_total else 0.0
                reasons = by_reason.get(task, {}) or {}
                reasons = {k: v for k, v in reasons.items() if k != "ok"}
                top = sorted(reasons.items(), key=lambda kv: kv[1], reverse=True)[:5]
                top_str = ", ".join([f"{k}:{v}" for k, v in top]) if top else "n/a"
                print(
                    f"[reward_diag]  - {task}: invalid_rate={t_rate:.3f} "
                    f"({t_invalid}/{t_total}) top={top_str}"
                )

        # KL spike dump
        kl = logs.get("kl", None)
        try:
            if kl is not None and float(kl) >= self.kl_spike_threshold:
                batch = reward_diag_last(state.global_step) or {}
                print(
                    f"[reward_diag] KL spike at step={state.global_step} kl={float(kl):.4f} batch={batch}"
                )
        except Exception:
            pass


diag_callback = RewardDiagnosticsCallback(kl_spike_threshold=5.0)

print("✓ Reward function 加载成功")

✓ Reward function 加载成功


## 5. 配置 GRPOTrainer

In [6]:
from trl import GRPOConfig, GRPOTrainer

# 快速验证模式：取消下面注释以使用短训练
# QUICK_VERIFY = True
QUICK_VERIFY = False  # 正式训练设为 False

# Eval 配置：eval_dataset 为空则自动禁用
DO_EVAL = False
EVAL_STEPS = 20
EVAL_BATCH_SIZE = 4  # 必须能整除 num_generations

config = GRPOConfig(
    output_dir="checkpoints/grpo_tsc_two_scenarios",

    # 批次配置
    per_device_train_batch_size=2,
    num_generations=4,  # Keep at 4 for proper GRPO
    gradient_accumulation_steps=4,

    # 生成配置
    max_completion_length=128,  
    temperature=0.7,
    top_p=0.9,
    top_k=40,
    use_vllm=False,

    # 训练配置
    learning_rate=2e-6,
    num_train_epochs=1,
    max_steps=20 if QUICK_VERIFY else -1,  # 快速验证：20步；正式训练：全epoch

    # GRPO 特定
    scale_rewards=True,

    # Eval (track real progress on held-out states)
    do_eval=DO_EVAL,
    eval_strategy="steps" if DO_EVAL else "no",
    eval_steps=EVAL_STEPS,
    eval_on_start=DO_EVAL,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,

    # 日志与保存
    logging_steps=5,
    save_steps=100,
    save_total_limit=3,

    # 优化器
    optim="adamw_torch",
    weight_decay=0.001,
    warmup_ratio=0.1,
    beta=0.01,             # KL 系数（0.0 默认不加载 ref）:contentReference[oaicite:1]{index=1}
    max_grad_norm=0.5,    # 梯度裁剪（防 KL spike）

    # 其他
    bf16=True,
    report_to="none",
    remove_unused_columns=False,
)

print(f"✓ GRPOConfig 配置完成 (模式: {'快速验证' if QUICK_VERIFY else '正式训练'})")
if QUICK_VERIFY:
    print(f"  - max_steps: {config.max_steps} (验证模式)")
else:
    print(f"  - max_steps: {config.max_steps} (全epoch)")

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 2 to the `num_generations` of 4
✓ GRPOConfig 配置完成 (模式: 正式训练)
  - max_steps: -1 (全epoch)


## 6. 创建 GRPOTrainer 并开始训练

In [ ]:
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    args=config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    reward_funcs=tsc_reward_fn,
    callbacks=[diag_callback],
)



print("✓ GRPOTrainer 创建成功")
print(f"训练样本数: {len(train_dataset)}")
print(f"评估样本数: {0 if eval_dataset is None else len(eval_dataset)}")
print(f"每 epoch steps: {len(trainer.get_train_dataloader())}")

print("\n" + "="*60)
print("开始 GRPO 训练")
print("="*60 + "\n")

trainer.train()

print("\n" + "="*60)
print("训练完成")
print("="*60)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 151644, 'pad_token_id': 151654}.


✓ GRPOTrainer 创建成功
训练样本数: 1674
评估样本数: 35
每 epoch steps: 1672

开始 GRPO 训练



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,674 | Num Epochs = 1 | Total steps = 418
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 17,596,416 of 511,629,184 (3.44% trained)


WARNING 01-19 22:59:25 [processor.py:215] vLLM has deprecated support for supporting different tokenizers for different LoRAs. By default, vLLM uses base model's tokenizer. If you are using a LoRA with its own tokenizer, consider specifying `--tokenizer [lora_path]` to use the LoRA tokenizer.

[DEBUG] Completion #1
原始输出:
{"extend": "是", "extend_sec": 28}

[tsc_reward_function] 初始化 spawn 进程池，workers=16，端口范围=20000-21600
Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / tsc_reward_fn / mean,rewards / tsc_reward_fn / std
5,0.039900,-2.284631,0.480154,17.950000,17.000000,19.000000,0.000000,17.950000,17.000000,19.000000,0,0,0,0,0,0.996547,-2.284631,1.765561
10,0.033600,-2.673727,0.368349,17.700000,17.200000,18.600000,0.000000,17.700000,17.200000,18.600000,No Log,No Log,No Log,No Log,No Log,0.839054,-2.673727,3.155528
15,0.036400,-3.156652,0.559691,17.900000,17.000000,19.000000,0.000000,17.900000,17.000000,19.000000,No Log,No Log,No Log,No Log,No Log,0.910482,-3.156652,3.720711
20,0.041300,-2.665663,0.885089,18.062500,17.200000,19.000000,0.000000,18.062500,17.200000,19.000000,No Log,No Log,No Log,No Log,No Log,1.031524,-2.665663,2.740815
25,0.034900,-2.715567,0.758484,17.812500,17.000000,19.000000,0.000000,17.812500,17.000000,19.000000,No Log,No Log,No Log,No Log,No Log,0.873310,-2.715567,2.761288
30,0.036900,-2.451863,0.771672,17.900000,17.200000,19.000000,0.000000,17.900000,17.200000,19.000000,No Log,No Log,No Log,No Log,No Log,0.922809,-2.451863,3.238644
35,0.028200,-2.959561,0.769093,17.550000,17.000000,19.000000,0.000000,17.550000,17.000000,19.000000,No Log,No Log,No Log,No Log,No Log,0.704161,-2.959561,2.827923
40,0.030200,-2.619528,0.537927,17.662500,17.000000,19.000000,0.000000,17.662500,17.000000,19.000000,No Log,No Log,No Log,No Log,No Log,0.755447,-2.619528,2.773231
45,0.034700,-2.189375,0.689579,17.837500,17.000000,19.000000,0.000000,17.837500,17.000000,19.000000,No Log,No Log,No Log,No Log,No Log,0.867572,-2.189375,3.173761
50,0.033600,-3.222269,1.144348,17.850000,17.000000,19.000000,0.000000,17.850000,17.000000,19.000000,No Log,No Log,No Log,No Log,No Log,0.839834,-3.222269,3.347281



[DEBUG] Completion #2
原始输出:
{"next_phase_id": 3, "green_sec": 56}


[DEBUG] Completion #3
原始输出:
{"next_phase_id": 1, "green_sec": 52}

[reward_diag] steps 0..5 invalid_rate=0.175 (14/80)
[reward_diag]  - extend_decision: invalid_rate=0.292 (14/48) top=extend_decision_final_green_out_of_bounds:10, extend_decision_extend_when_at_max_green:4
[reward_diag]  - signal_step: invalid_rate=0.000 (0/32) top=n/a
[reward_diag] steps 5..10 invalid_rate=0.050 (4/80)
[reward_diag]  - extend_decision: invalid_rate=0.111 (4/36) top=extend_decision_extend_when_at_max_green:3, extend_decision_final_green_out_of_bounds:1
[reward_diag]  - signal_step: invalid_rate=0.000 (0/44) top=n/a
[reward_diag] steps 10..15 invalid_rate=0.100 (8/80)
[reward_diag]  - extend_decision: invalid_rate=0.200 (8/40) top=extend_decision_final_green_out_of_bounds:4, extend_decision_extend_when_at_max_green:4
[reward_diag]  - signal_step: invalid_rate=0.000 (0/40) top=n/a
[reward_diag] steps 15..20 invalid_rate=0.225 (18/80)
[re

## 7. 保存最终模型

In [ ]:
final_output_dir = "checkpoints/grpo_tsc_two_scenarios_final"
trainer.save_model(final_output_dir)
tokenizer.save_pretrained(final_output_dir)
print(f"✓ 最终模型已保存到: {final_output_dir}")

## 8. 清理资源

In [ ]:
cleanup_global_pool()
print("✓ Simulator 池已清理")

## 9. 测试推理（可选）

In [ ]:
FastLanguageModel.for_inference(model)

test_sample = dataset[0]
test_prompt = test_sample["prompt"]

prompt_text = tokenizer.apply_chat_template(test_prompt, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)

print("测试生成:")
print("-" * 60)

outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
)

generated_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
print(generated_text)
print("-" * 60)